In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, average_precision_score, 
                             roc_curve, precision_recall_curve, auc, roc_auc_score, confusion_matrix)
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNClassifier
from matplotlib import font_manager
from matplotlib.font_manager import FontProperties  
import torch
from torch.utils.data import DataLoader, TensorDataset

In [3]:
# 读取数据
train = pd.read_csv('/root/autodl-fs/data/train_revise+45缩减到100特征 数量1000个 去掉三列和Name.csv')  

# 分离特征和标签
X = train.drop(['senolytic'], axis=1)
y = train['senolytic']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# 1. 加载 TabPFNClassifier 模型
model = TabPFNClassifier(device='cuda', model_file="/root/autodl-fs/Sq-TabPFN/tabpfn/增强微调模型/微调编码器+动态采样数据集/encoder_finetune_best_model.cpkt")

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# 检查predict_proba方法是否存在，以便获取概率
y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else model.decision_function(X_test)

Loading /root/autodl-fs/Sq-TabPFN/tabpfn/增强微调模型/微调编码器+动态采样数据集/encoder_finetune_best_model.cpkt
Loading model that can be used for inference only


TypeError: string indices must be integers

In [6]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
f1 = f1_score(y_test, y_pred, average='binary')
fpr, tpr, _ = roc_curve(y_test, y_proba)  # 计算假阳性率和真阳性率
roc_auc = auc(fpr, tpr) 
pr_precision, pr_recall, _ = precision_recall_curve(y_test, y_proba)
pr_auc = auc(pr_recall, pr_precision)

# 打印指标
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"PR AUC: {pr_auc:.4f}")

# 输出混淆矩阵
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Accuracy: 0.9150
Precision: 0.8077
Recall: 0.6364
F1 Score: 0.7119
ROC AUC: 0.8386
PR AUC: 0.6726
Confusion Matrix:
[[162   5]
 [ 12  21]]
